In [2]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import pandas as pd


def cluster(df, num_clusters, labels, trial_counter):
    """
    Clusters the df into num_clusters, returns labels with
    """
    #Fit df to num_clustersd and append cluster labels to df
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, init_size=600, random_state=0)
    kmeans = kmeans.fit(df.values)
    cluster_labels = kmeans.labels_
    df["Cluster"] = cluster_labels

    #Set All NAS delays
    df['NAS'] = labels["0"]

    #Get size and NAS ratio of each cluster and store in cluster array
    clusters = []
    for cluster in df["Cluster"].unique():
        tmp = df[df["Cluster"] == cluster]
        denominator = len(tmp.index)
        tmp = tmp[tmp["NAS"] != 0]
        numerator = len(tmp.index)
        clusters.append([cluster, (numerator / denominator), denominator])

    #sort clusters delay in descending order of NAS %
    clusters.sort(key = lambda tup: tup[1], reverse=True)

    #Accept clusters in descending order of NAS ratio until we reach 55% of the data
    num_accepted = 0
    accepted = []
    for entry in ratios:
        if(num_accepted/len(df.index) > 0.55):
            break
        accepted.append(entry[1])
        num_accepted += entry[2]
        ratios.remove(entry)
        
    #Reject all delays not in the accepted set
    for index, row in df.iterrows():
        if row['Cluster'] in accepted:
            pass
        else:
            row['NAS'] = 0
    #Add trial counter        
    df['Trial'] = trial_counter
    
    return df



def do_cluster(df):
    labels = pd.read_csv("NAS_train.csv")
    del labels['Unnamed: 0']
    trial = 1
    num_c = 100
    results = pd.DataFrame()
    while(num_c <= 500):
        if(trial == 1):
            results = cluster(df, num_c, labels, trial)
        else:
            results.append(cluster(df, num_c, labels, trial))
        num_c += 1
        trial += 1
    return results

In [30]:
df = pd.read_csv("X.csv")
del df['Unnamed: 0']
del df["OFFSET"]

remove = ['OP_UNIQUE_CARRIERWN', 'OP_UNIQUE_CARRIERAS',
       'OP_UNIQUE_CARRIERF9', 'OP_UNIQUE_CARRIERB6', 'OP_UNIQUE_CARRIERDL',
       'OP_UNIQUE_CARRIERUA', 'OP_UNIQUE_CARRIERAA', 'OP_UNIQUE_CARRIERNK',
       'OP_UNIQUE_CARRIEROO', 'OP_UNIQUE_CARRIERYX', 'OP_UNIQUE_CARRIER9E',
       'OP_UNIQUE_CARRIERVX']
df = df.drop(remove, axis=1)


results_a = do_cluster(df)
results_a.to_csv('ClustersA.csv')


remove = ['ORIGINSEA', 'ORIGINLAX', 'ORIGINMCO', 'ORIGINSFO',
       'ORIGINJFK', 'ORIGINATL', 'ORIGINDEN', 'ORIGINLAS', 'ORIGINDFW',
       'DESTDEN', 'DESTLAX', 'DESTJFK', 'DESTMCO', 'DESTLAS', 'DESTSEA',
       'DESTATL', 'DESTSFO', 'DESTDFW']
df = df.drop(remove, axis=1)

results_b = do_cluster(df)
results_b.to_csv('ClustersB.csv')



In [4]:
pd.read_csv("ClustersB.csv")

,Unnamed: 0,DEP_TIME^2,DEP_TIME^3,DEP_TIME,PGTM_x,PRCP_x,SNOW_x,SNWD_x,TAVG_x,TMAX_x,...,WT03_y,WT04_y,WT05_y,WT06_y,WT07_y,WT08_y,WT09_y,Cluster,NAS,Trial
0,0,2304324.0,3.497964e+09,1518.0,0.0,0.00,0.0,0.0,63.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78,0.0,401
1,1,1229881.0,1.363938e+09,1109.0,0.0,0.00,0.0,0.0,63.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,389,0.0,401
2,2,311364.0,1.737411e+08,558.0,0.0,0.00,0.0,0.0,63.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,406,0.0,401
3,3,549081.0,4.068690e+08,741.0,0.0,0.00,0.0,0.0,63.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,245,0.0,401
4,4,1731856.0,2.279122e+09,1316.0,0.0,0.00,0.0,0.0,57.0,65.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70,0.0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82707,82707,2712609.0,4.467667e+09,1647.0,0.0,0.03,0.0,0.0,72.0,78.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,118,62.0,401
82708,82708,537289.0,3.938328e+08,733.0,0.0,1.25,0.0,0.0,60.0,65.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,451,0.0,401
82709,82709,2941225.0,5.044201e+09,1715.0,0.0,1.25,0.0,0.0,60.0,65.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,107,0.0,401
82710,82710,4507129.0,9.568635e+09,2123.0,0.0,1.25,0.0,0.0,60.0,65.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,238,0.0,401
